In [ ]:
from operator import add
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, lit
from pyspark.sql.types import StructType, StructField, StringType

spark = SparkSession.builder.appName("pagerank_rdd").getOrCreate()

NUMBER_ITERATION = 2
NUMBER_PARTITION = 4

In [47]:
sc = spark.sparkContext
sc.textFile("../data/wikilink.csv")

def csv_reader(s):
    elements = s.split(';')
    return (elements[0], elements[1].split(' '))

data = sc.textFile("../data/wikilink.csv").map(csv_reader)

# Partitionner et persister l'adjacency (utile pour map-side join)
numParts = sc.defaultParallelism
data = data.partitionBy(numParts).persist()

In [37]:
def compute_contrib(children, rank):
    number_children = len(children)
    for c in children:
        yield c, rank / number_children

In [48]:
ranks = data.mapValues(lambda _:1.0)

for iteration in range(NUMBER_ITERATION):
    contrib = data.join(ranks).flatMap(lambda row: compute_contrib(row[1][0], row[1][1]))
    ranks = contrib.reduceByKey(add).mapValues(lambda rank : rank * 0.85 + 0.15)

In [49]:
ranks.collect()

[('<http://dbpedia.org/resource/Category:Gold_Standard_Laboratories_albums>',
  0.20666666666666667),
 ('<http://dbpedia.org/resource/Dance-punk>', 0.25170398009950246),
 ('<http://dbpedia.org/resource/Ian_Curtis>', 0.20666666666666667),
 ('<http://dbpedia.org/resource/Justin_Van_Der_Volgen>', 0.25170398009950246),
 ('<http://dbpedia.org/resource/Nic_Offer>', 0.25170398009950246),
 ('<http://dbpedia.org/resource/Post-punk_revival>', 0.20666666666666667),
 ('<http://dbpedia.org/resource/Stylus_Magazine>', 0.2644924242424242),
 ('<http://dbpedia.org/resource/Tyler_Pope>"', 0.20666666666666667),
 ('<http://dbpedia.org/resource/Abbreviation>', 0.16209615384615383),
 ('<http://dbpedia.org/resource/Berlin>', 0.1966628205128205),
 ('<http://dbpedia.org/resource/Booka_Shade>', 0.16209615384615383),
 ('<http://dbpedia.org/resource/Boozoo_Bajou>', 0.16209615384615383),
 ('<http://dbpedia.org/resource/Category:Electronic_music_record_labels>',
  0.16209615384615383),
 ('<http://dbpedia.org/resour